In [53]:
import pickle
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from statsmodels.formula.api import ols
import numpy as np
import statsmodels.api as sm
import researchpy as rp
import os
import re

In [46]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [47]:
root_dir = os.getcwd() + '\\pickles\\' # put all the pickle files to this directory
val_acc = []
avg_val_acc=[]
val_loss=[]
label_error=[]
image_noise=[]
labeler=[]
repetition=[]
for file in os.listdir(root_dir):
    file_name = root_dir + file
    filein = open(file_name, "rb")
    info = pickle.load(filein, encoding="utf8")
    val_acc.extend(info[14][0])
    s_bnn = 0
    for i in range(0, 10):
        s_bnn = s_bnn + val_acc[-5 * i - 1]
    average_valacc = s_bnn / 10
    avg_val_acc.append(average_valacc)
    val_loss.extend(info[16][0])
    label_error.append(re.findall('\d+', file)[9])
    image_noise.append(re.findall('\d+', file)[10])
    labeler.append(re.search(r'True|False', file).group())
    repetition.append(re.findall('\d+', file)[11])

    filein.close()

df = pd.DataFrame({'label_error':label_error,
                            'image_noise':image_noise,
                            'labeler':labeler, 
                            'val_acc':avg_val_acc})

In [48]:
rp.summary_cont(df.groupby(list(df.columns[:-1].values)))[df.columns[-1]]

N    Mean      SD      SE  95% Conf.  \
label_error image_noise labeler                                         
0           0           False    3  0.9316  0.0014  0.0008     0.9281   
                        True     3  0.9333  0.0116  0.0067     0.9045   
            40          False    3  0.9163  0.0109  0.0063     0.8892   
                        True     3  0.9187  0.0037  0.0021     0.9095   
40          0           False    3  0.8855  0.0043  0.0025     0.8749   
                        True     3  0.8957  0.0042  0.0024     0.8853   
            40          False    3  0.8618  0.0099  0.0057     0.8372   
                        True     3  0.8674  0.0095  0.0055     0.8439   

                                 Interval  
label_error image_noise labeler            
0           0           False      0.9352  
                        True       0.9620  
            40          False      0.9434  
                        True       0.9279  
40          0           False      0.8961  
                        True       0.9060  
            40          False      0.8864  
                        True       0.8910

In [49]:
model_string = "%s ~ "%df.columns[-1]+"".join(["C(%s)*"%i if i != df.columns[-1] else '' for i in df.columns])[:-1]

In [51]:
model = ols(model_string, data=df).fit()

In [57]:
sm.stats.anova_lm(model, typ=2)

,sum_sq,df,F,PR(>F)
C(label_error),0.013467,1.0,219.058609,9.337064e-11
C(image_noise),0.002507,1.0,40.786327,9.014189e-06
C(labeler),0.000146,1.0,2.380966,1.423673e-01
C(label_error):C(image_noise),0.000181,1.0,2.951486,1.050862e-01
C(label_error):C(labeler),0.000053,1.0,0.866235,3.658309e-01
C(image_noise):C(labeler),0.000006,1.0,0.090731,7.671293e-01
C(label_error):C(image_noise):C(labeler),0.000010,1.0,0.168558,6.868424e-01
Residual,0.000984,16.0,NaN,NaN
